In [ ]:
from pylab import *
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyfits
from astropy import constants
from astropy import units
from collections import OrderedDict

from matplotlib.ticker import NullFormatter  # useful for `logit` scale
from astropy.time import Time

from astropy.stats import LombScargle
#plt.style.use('seaborn-whitegrid')

%matplotlib inline
%config InlineBackend.figure_format='svg'
#%config InlineBackend.figure_format='retina'

# D_cm and Ledd

In [ ]:
from astropy.cosmology import FlatLambdaCDM,Planck13,Planck15,z_at_value
from astropy import units as u
import astropy
from astropy.cosmology import LambdaCDM
cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7)
#cosmo=FlatLambdaCDM(H0=73,Om0=0.3)
#D_cm=cosmo.luminosity_distance(z=0.042436).to(u.cm)
D_cm=7.23*u.Mpc.to(u.cm)
BH_mass=10**6.9*u.solMass
Ledd=1.26*10**38*BH_mass/u.solMass

In [ ]:
def set_ax_tick(ax):
    ax.xaxis.set_tick_params(which='major', size=10, width=2, direction='in', top='on',)
    ax.xaxis.set_tick_params(which='minor', size=5, width=2, direction='in', top='on')
    ax.yaxis.set_tick_params(which='major', size=10, width=2, direction='in', right='on')
    ax.yaxis.set_tick_params(which='minor', size=5, width=2, direction='in', right='on')

def set_ax_locator(ax,xma=1,xmi=0.2,yma=1,ymi=0.2):
    ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(xma))
    ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(xmi))
    ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(yma))
    ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(ymi))

# data

In [ ]:
data_xrt_fit_NGC1566 = pd.read_csv("/Users/brettlv/Downloads/wise_cl/xrt_fit_result/NGC1566_xrt_fit.txt",delimiter='\s+',header=0)  


data_ubb_NGC1566 = pd.read_csv("/Users/brettlv/Downloads/wise_cl/fig/NGC1566_ubb_sum.csv",delimiter='|',header=0)  
data_um2_NGC1566 = pd.read_csv("/Users/brettlv/Downloads/wise_cl/fig/NGC1566_um2_sum.csv",delimiter='|',header=0)  


data_uuu_NGC1566 = pd.read_csv("/Users/brettlv/Downloads/wise_cl/fig/NGC1566_uuu_sum.csv",delimiter='|',header=0)  
data_uvv_NGC1566 = pd.read_csv("/Users/brettlv/Downloads/wise_cl/fig/NGC1566_uvv_sum.csv",delimiter='|',header=0)  

data_uw1_NGC1566 = pd.read_csv("/Users/brettlv/Downloads/wise_cl/fig/NGC1566_uw1_sum.csv",delimiter='|',header=0)  
data_uw2_NGC1566 = pd.read_csv("/Users/brettlv/Downloads/wise_cl/fig/NGC1566_uw2_sum.csv",delimiter='|',header=0)  


data_ASAS_NGC1566 = pd.read_csv("/Users/brettlv/Downloads/NGC1566/AP36757554_ASASSN-V J042000.52-545616.7 (65.00215, -54.93797) .csv",header=0) 
data_ASAS_NGC1566['hmjd']= data_ASAS_NGC1566['hjd']- 2400000.5


## good fit

In [ ]:
data_xrt_goodfit_NGC1566=data_xrt_fit_NGC1566[data_xrt_fit_NGC1566['chi']<1.5]
data_xrt_goodfit_NGC1566=data_xrt_goodfit_NGC1566[data_xrt_goodfit_NGC1566['chi']>0.7]

In [ ]:
data_xrt_goodfit_NGC1566.to_csv('/Users/brettlv/Downloads/NGC1566/NGC1566_goodfit.csv')

## F-Gamma good-fit

In [ ]:
def fluxtoeddrate(x):
    y = 4*np.pi*D_cm**2*x/Ledd
    return y

def eddratetoflux(x):    
    y = x*Ledd*(4*np.pi*D_cm**2)**(-1)
    return y 


fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)

set_ax_tick(ax)

ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.5))
ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.25))

secax = ax.secondary_xaxis('top', functions=(fluxtoeddrate,eddratetoflux))
secax.set_xlabel(r'$L_{1-10keV}/L_{Edd}}$')
secax.xaxis.set_tick_params(which='major', size=10, width=2, direction='out')
secax.xaxis.set_tick_params(which='minor', size=5, width=2, direction='out')

lower_error_f = data_xrt_goodfit_NGC1566['xflux_lo']#1-10keV
upper_error_f = data_xrt_goodfit_NGC1566['xflux_up']
lower_error_g = data_xrt_goodfit_NGC1566['gamma_lo']
upper_error_g = data_xrt_goodfit_NGC1566['gamma_up']
asymmetric_error_f = [lower_error_f, upper_error_f]
asymmetric_error_g = [lower_error_g, upper_error_g]



ax.set_xlabel(r'$F_{1-10keV}$')
ax.set_ylabel(r'$\Gamma$')
ax.semilogx()
ax.errorbar(data_xrt_goodfit_NGC1566['xflux'],data_xrt_goodfit_NGC1566['gamma'],xerr=asymmetric_error_f,yerr=asymmetric_error_g,fmt='.',label='F-gamma') 

plt.rcParams['font.size'] = 18
plt.rcParams['axes.linewidth'] = 2
plt.savefig('/Users/brettlv/Downloads/NGC1566/NGC1566_goodfit_F-Gamma.png',dpi=400)

In [ ]:
import plotly_express as px
px.scatter?

In [ ]:

import plotly_express as px

fig = px.scatter(data_xrt_goodfit_NGC1566, x='xflux',error_x='xflux_up',
                 error_x_minus='xflux_lo',y='gamma',error_y='gamma_up',error_y_minus='gamma_lo',
                 hover_name="obs_time",hover_data=['obs_time','chi','obs_id'],
                 size_max=15,color='obs_time')


fig.update_xaxes(tickformat= "0.1e",nticks=4,type="log",title_text="flux")
fig.update_yaxes(tickformat= "0.2f",nticks=10,type="linear",title_text="gamma")
fig.update_layout(title_text="", width = 800,height = 600)
#fig.update_xaxes(range=[np.log10(2e-4),np.log10(1e-2)])
#fig.update_yaxes(range=[1.0,2.5])
fig.show()
#fig.write_image('xray-paper-g-eddrate.gif')
#fig.write_image('xray-paper-g-eddrate.png')
fig.write_html('/Users/brettlv/Downloads/NGC1566/g-f-with-time.html')

In [ ]:
min(data_xrt_goodfit_NGC1566['xflux']),max(data_xrt_goodfit_NGC1566['xflux'])

In [ ]:
import plotly_express as px

fig = px.scatter(data_xrt_goodfit_NGC1566, x='xflux',error_x='xflux_up',
                 error_x_minus='xflux_lo',y='gamma',error_y='gamma_up',error_y_minus='gamma_lo',
                 hover_name="obs_time",hover_data=['obs_time','chi','obs_id'],
                 size_max=15,animation_frame='obs_time')


fig.update_xaxes(tickformat= "0.1e",nticks=3,type="log",title_text=r"$F_{1-10 keV}$")
fig.update_yaxes(tickformat= "0.2f",nticks=5,type="linear",title_text=r"$\Gamma$")
fig.update_layout(title_text="", width = 600,height = 600)
fig.update_xaxes(range=[np.log10(1e-12),np.log10(2e-10)])
fig.update_yaxes(range=[1.0,2.5])
fig.show()
#fig.write_image('xray-paper-g-eddrate.gif')
#fig.write_image('xray-paper-g-eddrate.png')
fig.write_html('/Users/brettlv/Downloads/NGC1566/g-f-with-time-animation.html')

A(E) \prop E^{-\Gamma}
F_v \prop v^{-\alpha}
\Gamma=\alpha+1

## XRT_good_fit

In [ ]:
v_1keV=2.418e17 #Hz
alpha_goodfit=data_xrt_goodfit_NGC1566['gamma']-1
v_2keV =2*v_1keV
data_xrt_goodfit_NGC1566['F_{1keV}']=data_xrt_goodfit_NGC1566['xflux']*(1-alpha_goodfit)/(10**(1-alpha_goodfit)-1)/v_1keV
data_xrt_goodfit_NGC1566['F_{2keV}']=2**(-alpha_goodfit)*data_xrt_goodfit_NGC1566['F_{1keV}']
data_xrt_goodfit_NGC1566['F_{4keV}']=4**(-alpha_goodfit)*data_xrt_goodfit_NGC1566['F_{1keV}']
data_xrt_goodfit_NGC1566['F_{6keV}']=6**(-alpha_goodfit)*data_xrt_goodfit_NGC1566['F_{1keV}']
data_xrt_goodfit_NGC1566['F_{8keV}']=8**(-alpha_goodfit)*data_xrt_goodfit_NGC1566['F_{1keV}']
data_xrt_goodfit_NGC1566['F_{10keV}']=10**(-alpha_goodfit)*data_xrt_goodfit_NGC1566['F_{1keV}']

alpha=gamma-1

## cal_mono_flux

In [ ]:
v_1keV=2.418e17 #Hz
v2=k*v1
F_{v}=m*v**(-alpha)
F_{v1}=F_{v1-v2}*(1-alpha)/(k**(1-alpha)-1)/v1
F_{2keV}=F_{2-10keV}*(1-alpha)/(5**(1-g)-1)/v_{2keV}
F_{1keV}=F_{1-10keV}*(1-alpha)/(10**(1-g)-1)/v_{1keV}
F_{2keV}=2**(-alpha)*F_{1keV}


np.sum(data_xrt_goodfit_NGC1566[data_xrt_goodfit_NGC1566.columns[2]]==1.0)

## ASAS-SN -48.8

In [ ]:
1Jy =1.0e-26 W m-2 Hz-1

= 1.0e-23 erg s-1 cm-2 Hz-1
 
    

In [ ]:
data_ASAS_NGC1566['flux']=(data_ASAS_NGC1566['flux (mJy)']-48.8)*1.0e-26#*1.0e-3*1.0e7*1.0e-4

In [ ]:
for i,j in enumerate(data_ubb_NGC1566.columns):
    print(i,j)

## UV

In [ ]:
UV_namelist=['v','b','u','uvw1','uvm2','uvw2']
UV_datalist=[data_uvv_NGC1566,data_ubb_NGC1566,data_uuu_NGC1566,data_uw1_NGC1566,data_um2_NGC1566,data_uw2_NGC1566  ]
UV_lambda_list=[5.468E-07,4.392E-07,3.465E-07,2.600E-07,2.246E-07,1.928E-07]#

In [ ]:
data_uvv_NGC1566.columns

In [ ]:
for i in UV_datalist:
    print(i.columns[4])
    i['flux']=10**((-i[i.columns[4]]-48.6)/2.5)

In [ ]:
data_uw1_NGC1566['flux']

In [ ]:
sed_neowise_w1mpro_data

## wise

In [ ]:
data_allwise_epoch=pd.read_csv('/Users/brettlv/Downloads/wise_cl/NGC1566_multi_epoch.csv')
data_neowise=pd.read_csv('/Users/brettlv/Downloads/wise_cl/NGC1566_neo.csv')

In [ ]:
data_neowise_select=data_neowise.loc[:, ['w1mpro', 'w1sigmpro',
         'w2mpro', 'w2sigmpro','w1mpro_allwise', 'w1sigmpro_allwise', 'w2mpro_allwise',
        'w2sigmpro_allwise', 'w3mpro_allwise', 'w3sigmpro_allwise',
        'w4mpro_allwise', 'w4sigmpro_allwise','mjd']]


In [ ]:
fig, axs = plt.subplots(1, 1,figsize=(8,8))
axs.set_title('NGC1566_wise')

delta_m_wise=[2.699,3.339,5.174,6.620]
delta_m_neowise=[2.699,3.339,2.699,3.339,5.174,6.620]


for i in range(4):
    name_mjd='mjd'
    name_band=data_allwise_epoch.columns[i*3+8]
    name_err=data_allwise_epoch.columns[i*3+9]
    print(name_mjd,name_band,name_err,delta_m_wise[i])
    data_band=data_allwise_epoch[data_allwise_epoch[name_mjd].notnull()]
    data_band=data_band[data_band[name_band].notnull()]
    #mag_wise_data=pd.DataFrame({'t':data_band[name_mjd],'flux':data_band[name_band],'err':data_band[name_err]},index=None,columns=['t','flux','err'])    
    dcf_wise_data=pd.DataFrame({'t':data_band[name_mjd],'log_flux':(-48.6-data_band[name_band]-delta_m_wise[i])/2.5,'err':data_band[name_err]/2.5},index=None,columns=['t','log_flux','err'])    
    dcf_wise_data['flux']=10**dcf_wise_data['log_flux']
    dcf_wise_data=dcf_wise_data.sort_values('t')
    dcf_wise_data.to_csv('/Users/brettlv/Downloads/NGC1566/sed_wise_multi_%s_data.csv'%name_band,index=False,header=False)          
    axs.errorbar(dcf_wise_data['t'],dcf_wise_data['log_flux'],dcf_wise_data['err'],fmt='.',label=name_band+'multi_epoch')
    print(max(dcf_wise_data['t']))
    

for i in range(6):
    name_mjd='mjd'
    name_band=data_neowise_select.columns[i*2]
    name_err=data_neowise_select.columns[i*2+1]
    print(name_mjd,name_band,name_err,delta_m_neowise[i])     
    #mag_neowise_data=pd.DataFrame({'t':data_neowise[name_mjd],'flux':data_neowise[name_band],'err':data_neowise[name_err]},index=None,columns=['t','flux','err'])   
    data_new=data_neowise_select[data_neowise_select[name_mjd].notnull()]
    data_new=data_new[data_new[name_band].notnull()]
    
    dcf_neowise_data=pd.DataFrame({'t':data_new[name_mjd],'log_flux':(-48.6-data_new[name_band]-delta_m_neowise[i])/2.5,'err':data_new[name_err]/2.5},index=None,columns=['t','log_flux','err'])
    dcf_neowise_data['flux']=10**(dcf_neowise_data['log_flux'])
    dcf_neowise_data=dcf_neowise_data.sort_values('t')
    dcf_neowise_data.to_csv('/Users/brettlv/Downloads/NGC1566/sed_neowise_%s_data.csv'%name_band,index=False,header=False,)      
    axs.errorbar(dcf_neowise_data['t'],dcf_neowise_data['log_flux'],dcf_neowise_data['err'],fmt='.',label=name_band+'neo')
    print(max(dcf_neowise_data['t']))
    
axs.legend(loc=[1.0,0.7])
#axs.invert_yaxis() 
axs.set_ylabel('log_F')    
#axs.set_ylabel('magnitude(AB)')
axs.set_xlabel('mjd')
plt.tight_layout()
plt.savefig('/Users/brettlv/Downloads/NGC1566/wise_lc_sed.png',dpi=400)

In [ ]:
print(data_xrt_goodfit_NGC1566.columns)

## sim mjd

In [ ]:
data_uvv_NGC1566.columns

In [ ]:
data_xrt_goodfit_NGC1566.columns

## alpha_ox

In [ ]:
data_uw1_NGC1566.columns

In [ ]:
data_xrt_goodfit_NGC1566.columns

In [ ]:
v_2keV 

In [ ]:
print(data_uw1_NGC1566.columns[3],data_uw1_NGC1566.columns[6])

In [ ]:
print(data_xrt_goodfit_NGC1566.columns[1])

In [ ]:
f_uw1=(constants.c/(2600*units.AA)).to(u.Hz).value

def get_alpha_ox_mjd(data_uw1_NGC1566,data_xrt_goodfit_NGC1566,mjd):
    mjd_near_i=np.argmin(abs(data_uw1_NGC1566[data_uw1_NGC1566.columns[3]]-mjd))
    mjd_near=data_uw1_NGC1566[data_uw1_NGC1566.columns[3]][mjd_near_i]    
    if abs(mjd_near-mjd)<0.01:
        v_f_uw1=data_uw1_NGC1566[data_uw1_NGC1566.columns[6]][mjd_near_i] *1.0e-26*f_uw1       
        
    mjd_near_i=np.argmin(abs(data_xrt_goodfit_NGC1566[data_xrt_goodfit_NGC1566.columns[1]]-mjd))
    mjd_near=data_xrt_goodfit_NGC1566[data_xrt_goodfit_NGC1566.columns[1]][mjd_near_i] 
    
    if abs(mjd_near-mjd)<0.01:
        v_f_2keV=data_xrt_goodfit_NGC1566['F_{2keV}'][mjd_near_i]*v_2keV        
    alpha_ox=1-(np.log10(v_f_uw1)-np.log10(v_f_2keV))/(np.log10(f_uw1)-np.log10(v_2keV))    
    
    return alpha_ox, v_f_uw1,v_f_2keV



In [ ]:
def get_simultaneous_mjd(uvot,xray_append,timebin=1):
    t_uv=[]
    t_xray=[]
    print(uvot.columns[3],xray_append.columns[1])
    for i in uvot[uvot.columns[3]]:
        for j in xray_append[xray_append.columns[1]]:
            if abs(i-j)<timebin:
                t_uv.append(i)
                t_xray.append(j)                
    return np.array(t_uv),np.array(t_xray)

t_uw1,t_xray=get_simultaneous_mjd(data_uw1_NGC1566,data_xrt_goodfit_NGC1566,timebin=0.01)
t_uw1.sort()
print(len(t_uw1))


In [ ]:
fig = plt.figure(figsize=(8,6),dpi=400)
ax = fig.add_subplot(111)

for mjd in t_uw1:
    alpha_ox, v_f_uw1,v_f_2keV=get_alpha_ox_mjd(data_uw1_NGC1566,data_xrt_goodfit_NGC1566,mjd)
    ax.scatter(4*np.pi*D_cm**2*v_f_uw1/f_uw1, alpha_ox)
    
ax.xaxis.set_tick_params(which='major', size=10, width=2, direction='in', top='on',)
ax.xaxis.set_tick_params(which='minor', size=7, width=2, direction='in', top='on')
ax.yaxis.set_tick_params(which='major', size=10, width=2, direction='in', right='on')
ax.yaxis.set_tick_params(which='minor', size=7, width=2, direction='in', right='on')
#ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(100))
#ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(10))
ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.1))
ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.01))
ax.semilogx()
ax.set_xlabel(r'$L_{\mathregular{2600 \AA}}$ ($erg\,s^{-1}\,Hz^{-1}$)')
ax.set_ylabel(r'$\mathregular{\alpha_{OX}}$')


handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(1.2, 0.8),ncol=1,loc='right',fontsize=10)


ax.grid(alpha=0.1,which='major', linestyle='--', linewidth=1)
plt.subplots_adjust(hspace = 0.0)
plt.savefig('/Users/brettlv/Downloads/NGC1566/NGC1566_alpha_ox_L_uw1.png', dpi=300, transparent=False, bbox_inches='tight')


In [ ]:
fig = plt.figure(figsize=(8,6),dpi=400)
ax = fig.add_subplot(111)


#ax.set_title('SED of Mrk1018')
#ax.semilogy()
ax.semilogx()
#ax.set_xlabel(r'$F_{\mathregular{2\,keV}}$ ($erg\,s^{-1}\,cm^{-2}$)')
ax.set_xlabel(r'$L_{\mathregular{2\,keV}}$ ($erg\,s^{-1}\,Hz^{-1}$)')
ax.set_ylabel(r'$\mathregular{\alpha_{OX}}$')

for mjd in t_uw1:
    alpha_ox, v_f_uw1,v_f_2keV=get_alpha_ox_mjd(data_uw1_NGC1566,data_xrt_goodfit_NGC1566,mjd)
    ax.scatter(4*np.pi*D_cm**2*v_f_2keV/v_2keV, alpha_ox)
 
    
ax.xaxis.set_tick_params(which='major', size=10, width=2, direction='in', top='on',)
ax.xaxis.set_tick_params(which='minor', size=7, width=2, direction='in', top='on')
ax.yaxis.set_tick_params(which='major', size=10, width=2, direction='in', right='on')
ax.yaxis.set_tick_params(which='minor', size=7, width=2, direction='in', right='on')
#ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(100))
#ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(10))
ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.1))
ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.01))


handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(1.2, 0.8),ncol=1,loc='right',fontsize=10)


ax.grid(alpha=0.1,which='major', linestyle='--', linewidth=1)
plt.subplots_adjust(hspace = 0.0)
plt.savefig('/Users/brettlv/Downloads/NGC1566/NGC1566_alpha_ox_L_2keV.png', dpi=300, transparent=False, bbox_inches='tight')
# Show figure
plt.show()


In [ ]:
t_uvv,t_xray=get_simultaneous_mjd(data_uvv_NGC1566,data_xrt_goodfit_NGC1566,timebin=0.1)
t_uvv.sort()
print(len(t_uvv))
for i in t_uvv:
    print('%.2f'%i,end=' ,')

In [ ]:
#swift
mjd_list=[54467.73,55381.84,58330.99,58483.19,58681.29,58695.23]

In [ ]:
#wise
mjd_list=[55212,55400,55579,56677,56866,57039,57228,57400,57593,57761,58123,58323,58483,58689]


In [ ]:
t_uvv,t_xray=get_simultaneous_mjd(data_uvv_NGC1566,data_xrt_goodfit_NGC1566,timebin=0.1)
for i in t_uvv:
    print('%.2f'%i)

data_rebin_neowise_w1[0]

In [ ]:
55212,55400,55579,56677,56866,57039,57228,57400,57593,57761,58123,58323,58483,58689
54467,55370,55375,55381,58330,58338
58339
58339
58340
58340

In [ ]:
data_rebin_neowise_w1=pd.read_csv('/Users/brettlv/Downloads/NGC1566/rebin_dcf_neowise_w1.csv',header=None)
data_rebin_neowise_w2=pd.read_csv('/Users/brettlv/Downloads/NGC1566/rebin_dcf_neowise_w2.csv',header=None)

In [ ]:
plt.scatter(data_rebin_neowise_w1[0],data_rebin_neowise_w1[0])

plt.scatter(sed_multiwise_w1mpro_data[0],sed_multiwise_w1mpro_data[0])

In [ ]:
t_uvv.size

# get sed data

data_xrt_goodfit_NGC1566['obs_time']

for l in UV_lambda_list:
    v_i=(constants.c/l/units.m).to(units.Hz).value
    print(v_i)

for i in data_xrt_goodfit_NGC1566['obs_time']:
    print(i)

In [ ]:
sed_neowise_w1mpro_data=pd.read_csv('/Users/brettlv/Downloads/NGC1566/sed_neowise_w1mpro_data.csv',header=None)
sed_neowise_w2mpro_data=pd.read_csv('/Users/brettlv/Downloads/NGC1566/sed_neowise_w2mpro_data.csv',header=None)
sed_neowise_w3mpro_data=pd.read_csv('/Users/brettlv/Downloads/NGC1566/sed_neowise_w3mpro_allwise_data.csv',header=None)
sed_neowise_w4mpro_data=pd.read_csv('/Users/brettlv/Downloads/NGC1566/sed_neowise_w4mpro_allwise_data.csv',header=None)
sed_multiwise_w1mpro_data=pd.read_csv('/Users/brettlv/Downloads/NGC1566/sed_wise_multi_w1mpro_ep_data.csv',header=None)
sed_multiwise_w2mpro_data=pd.read_csv('/Users/brettlv/Downloads/NGC1566/sed_wise_multi_w2mpro_ep_data.csv',header=None)
sed_multiwise_w3mpro_data=pd.read_csv('/Users/brettlv/Downloads/NGC1566/sed_wise_multi_w3mpro_ep_data.csv',header=None)
sed_multiwise_w4mpro_data=pd.read_csv('/Users/brettlv/Downloads/NGC1566/sed_wise_multi_w4mpro_ep_data.csv',header=None)

In [ ]:
neowise_namelist=['w1mpro','w2mpro','w3mpro_allwise','w4mpro_allwise']
neowise_lambdalist=[3.400E-06,4.600E-06,1.200E-05,2.200E-05]
neowise_datalist=[sed_neowise_w1mpro_data,sed_neowise_w2mpro_data,sed_neowise_w3mpro_data,sed_neowise_w4mpro_data]
multiwise_namelist=['w1mpro_ep','w2mpro_ep','w3mpro_ep','w4mpro_ep']
multiwise_datalist=[sed_multiwise_w1mpro_data,sed_multiwise_w2mpro_data,sed_multiwise_w3mpro_data,sed_multiwise_w4mpro_data]
wise_lambdalist=[3.400E-06,4.600E-06,1.200E-05,2.200E-05]

In [ ]:
for l in wise_lambdalist:
    v_i=(constants.c/l/units.m).to(units.Hz).value
    print('{:.2e}'.format(v_i))

In [ ]:
for l in UV_lambda_list:
    v_i=(constants.c/l/units.m).to(units.Hz).value
    print('{:.2e}'.format(v_i))

## plot_vFv-v

In [ ]:
1Jy= 1e-26 W m-2Hz-1
   = 1e-26 J/s m-2 Hz-1
   = 1e-19 erg/s m-2 Hz-1
   = 1e-23 erg/s cm-2 Hz-1

## based on wise

In [ ]:
#mjd_list=[55212,55400,55579,56677,56866,57039,57228,57400,57593,57761,58123,58323,58483,58689]
mjd_list=[55212,55400,55579,56677,56866,57039,57228,57400,57593,57761,58123,58323,58483,58689]

markers_pd=['*','.','+','d','h','v','^','x','>','1','2','3','4','8','p','P','<','H','X','D','|','_']
colors_pd=['red','green','blue','yellow','lightblue','cyan','tan','gold','sandybrown','olivedrab','chartreuse','darkgreen','m','skyblue','violet','crimson','purple','magenta','lime','turquoise','lightcyan','teal','indigo']
marker_dict = dict(zip(mjd_list, markers_pd[:len(mjd_list)]))
colors_8 = cm.get_cmap('tab10', 8)
colors_pd_8=[colors_8(0),colors_8(1),colors_8(2),colors_8(3),colors_8(4),colors_8(5),colors_8(6),colors_8(7),]
color_dict=dict(zip(mjd_list, colors_pd[:len(mjd_list)]))

def plot_xrt_sed_time(mjd,data_xrt_goodfit_NGC1566,marker,color='r'):
    mjd_near_i=np.argmin(abs(data_xrt_goodfit_NGC1566['obs_time']-mjd))
    mjd_near=data_xrt_goodfit_NGC1566['obs_time'][mjd_near_i]
    if abs(mjd_near-mjd)<10:
        print('xrt',mjd_near)
        for i in range(5):
            #print(i+1,data_xrt_goodfit_NGC1566.columns[i+10])
            v_i=(i+1)*2*v_1keV
            f_v_i=data_xrt_goodfit_NGC1566[data_xrt_goodfit_NGC1566.columns[i+10]][mjd_near_i]
            print("%10.3e"% (v_i),"%10.3e"% (f_v_i))
            axs.scatter(v_i,v_i*f_v_i,marker=marker,label=mjd,color=color)
            #axs.text(v_i,v_i*f_v_i,str((i+1)*2))
            
            
def plot_uvot_sed_time(mjd,UV_namelist,UV_datalist,UV_lambda_list,marker,color='r'):
    for i,j,l in zip(UV_namelist,UV_datalist,UV_lambda_list):
        mjd_near_i=np.argmin(abs(j[j.columns[3]]-mjd))
        mjd_near=j[j.columns[3]][mjd_near_i]           
        if abs(mjd_near-mjd)<10:
            v_i=(constants.c/l/units.m).to(units.Hz).value
            f_v_i=j[j.columns[-1]][mjd_near_i]  
            print(i,mjd_near)
            print("%10.3e"% (v_i),"%10.3e"% (f_v_i))
            axs.scatter(v_i,v_i*f_v_i,marker=marker,label=mjd,color=color) 
            #axs.text(v_i,v_i*f_v_i,i)

def plot_wise_sed_time(mjd,wise_namelist,wise_datalist,wise_lambdalist,marker,color='r'):
    for i,j,l in zip(wise_namelist,wise_datalist,wise_lambdalist):
        mjd_near_i=np.argmin(abs(j[j.columns[0]]-mjd))
        mjd_near=j[j.columns[0]][mjd_near_i]           
        if abs(mjd_near-mjd)<2:
            print(i,mjd_near)
            v_i=(constants.c/l/units.m).to(units.Hz).value
            f_v_i=j[j.columns[3]][mjd_near_i]
            print("%10.3e"% (v_i),"%10.3e"% (f_v_i))
            axs.scatter(v_i,v_i*f_v_i,marker=marker,label=mjd,color=color) 
            #axs.text(v_i,v_i*f_v_i,i[:2])
            
def plot_mjd_sed(mjd,marker,color):
    plot_uvot_sed_time(mjd,UV_namelist,UV_datalist,UV_lambda_list,marker,color)
    plot_xrt_sed_time(mjd,data_xrt_goodfit_NGC1566,marker,color)
    if mjd < 56000:
        plot_wise_sed_time(mjd,multiwise_namelist,multiwise_datalist,wise_lambdalist,marker,color)                
    else:
        plot_neowise_sed_time(mjd,neowise_namelist,neowise_datalist,neowise_lambdalist,marker,color)
          

fig, axs = plt.subplots(1, 1,figsize=(8,6))
axs.set_title('NGC1566_SED')
axs.semilogy()
axs.semilogx()
axs.set_ylabel(r'$\nu F_{\nu}$ ($erg s^{-1} cm^{-2}$)')
axs.set_xlabel('Hz')

for i in mjd_list:
    marker=marker_dict[i]
    color=color_dict[i]
    plot_mjd_sed(i,marker,color)

    
axs.xaxis.set_tick_params(which='major', size=10, width=2, direction='in', top='on')
axs.xaxis.set_tick_params(which='minor', size=7, width=2, direction='in', top='on')
axs.yaxis.set_tick_params(which='major', size=10, width=2, direction='in', right='on')
axs.yaxis.set_tick_params(which='minor', size=7, width=2, direction='in', right='on')
#axs.xaxis.set_major_locator(mpl.ticker.MultipleLocator(100))
#axs.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(10))
    
#axs.set_xlim(6.0e13,1.0e19)    
    
handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))    
axs.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(1.2, 0.7),ncol=1,loc='right')    
axs.grid(alpha=0.1,which='major', linestyle='--', linewidth=1)
plt.tight_layout()
plt.savefig('/Users/brettlv/Downloads/NGC1566/NGC1566_sed_based_on_wise.png',dpi=400)

## based on Swift

In [ ]:
#mjd_list=[54467,55370,58330,58350,58380,58483,58695,58829]
mjd_list=[54467.73,55381.84,56930.51,56950.76,58323.93,58330.99,58483.19,58681.29,5848358695.23]

markers_pd=['*','.','+','d','h','v','^','x','>','1','2','3','4','8','p','P','<','H','X','D','|','_']
colors_pd=['red','green','blue','yellow','lightblue','cyan','tan','gold','sandybrown','olivedrab','chartreuse','darkgreen','m','skyblue','violet','crimson','purple','magenta','lime','turquoise','lightcyan','teal','indigo']
marker_dict = dict(zip(mjd_list, markers_pd[:len(mjd_list)]))
colors_8 = cm.get_cmap('tab10', 8)
colors_pd_8=[colors_8(0),colors_8(1),colors_8(2),colors_8(3),colors_8(4),colors_8(5),colors_8(6),colors_8(7),]
color_dict=dict(zip(mjd_list, colors_pd[:len(mjd_list)]))

def plot_xrt_sed_time(mjd,data_xrt_goodfit_NGC1566,marker,color='r'):
    mjd_near_i=np.argmin(abs(data_xrt_goodfit_NGC1566['obs_time']-mjd))
    mjd_near=data_xrt_goodfit_NGC1566['obs_time'][mjd_near_i]
    if abs(mjd_near-mjd)<0.5:
        print('xrt',mjd_near)
        for i in range(5):
            #print(i+1,data_xrt_goodfit_NGC1566.columns[i+10])
            v_i=(i+1)*2*v_1keV
            f_v_i=data_xrt_goodfit_NGC1566[data_xrt_goodfit_NGC1566.columns[i+10]][mjd_near_i]
            #print("%10.3e"% (v_i),"%10.3e"% (f_v_i))
            axs.scatter(v_i,v_i*f_v_i,marker=marker,label=mjd,color=color)
            #axs.text(v_i,v_i*f_v_i,str((i+1)*2))
            
            
def plot_uvot_sed_time(mjd,UV_namelist,UV_datalist,UV_lambda_list,marker,color='r'):
    for i,j,l in zip(UV_namelist,UV_datalist,UV_lambda_list):
        mjd_near_i=np.argmin(abs(j[j.columns[3]]-mjd))
        mjd_near=j[j.columns[3]][mjd_near_i]           
        if abs(mjd_near-mjd)<1.0:
            v_i=(constants.c/l/units.m).to(units.Hz).value
            f_v_i=j[j.columns[-1]][mjd_near_i]  
            print(i,mjd_near)
            #print("%10.3e"% (v_i),"%10.3e"% (f_v_i))
            axs.scatter(v_i,v_i*f_v_i,marker=marker,label=mjd,color=color) 
            #axs.text(v_i,v_i*f_v_i,i)

def plot_wise_sed_time(mjd,wise_namelist,wise_datalist,wise_lambdalist,marker,color='r'):
    for i,j,l in zip(wise_namelist,wise_datalist,wise_lambdalist):
        mjd_near_i=np.argmin(abs(j[j.columns[0]]-mjd))
        mjd_near=j[j.columns[0]][mjd_near_i]           
        if abs(mjd_near-mjd)<20:
            print(i,mjd_near)
            v_i=(constants.c/l/units.m).to(units.Hz).value
            f_v_i=j[j.columns[3]][mjd_near_i]
            print(i,"%10.3e"% (v_i),"%10.3e"% (f_v_i))
            axs.scatter(v_i,v_i*f_v_i,marker=marker,label=mjd,color=color) 
            #axs.text(v_i,v_i*f_v_i,i[:2])
            
def plot_mjd_sed(mjd,marker,color):
    plot_uvot_sed_time(mjd,UV_namelist,UV_datalist,UV_lambda_list,marker,color)
    plot_xrt_sed_time(mjd,data_xrt_goodfit_NGC1566,marker,color)
    if mjd < 56000:
        plot_wise_sed_time(mjd,multiwise_namelist,multiwise_datalist,wise_lambdalist,marker,color)                
    else:
        plot_wise_sed_time(mjd,neowise_namelist,neowise_datalist,wise_lambdalist,marker,color)
          

fig, axs = plt.subplots(1, 1,figsize=(8,8))
axs.set_title('NGC1566_SED')
axs.semilogy()
axs.semilogx()
axs.set_ylabel(r'$\nu F_{\nu}$ ($erg s^{-1} cm^{-2}$)')
axs.set_xlabel('Hz')

for i in mjd_list:
    marker=marker_dict[i]
    color=color_dict[i]
    plot_mjd_sed(i,marker,color)

    
axs.xaxis.set_tick_params(which='major', size=10, width=2, direction='in', top='on')
axs.xaxis.set_tick_params(which='minor', size=7, width=2, direction='in', top='on')
axs.yaxis.set_tick_params(which='major', size=10, width=2, direction='in', right='on')
axs.yaxis.set_tick_params(which='minor', size=7, width=2, direction='in', right='on')
#axs.xaxis.set_major_locator(mpl.ticker.MultipleLocator(100))
#axs.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(10))
    
#axs.set_xlim(6.0e13,1.0e19)    
    
handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))    
axs.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(1.2, 0.7),ncol=1,loc='right')    
axs.grid(alpha=0.1,which='major', linestyle='--', linewidth=1)
plt.tight_layout()
plt.savefig('/Users/brettlv/Downloads/NGC1566/NGC1566_sed_based_on_swift.png',dpi=400)

In [ ]:
1.264/2246,1.126/1928

In [ ]:
10**((-48.6-9.039)/2.5)* (constants.c/(3.400E-06*units.m)).to(units.Hz).value

In [ ]:
(constants.c/(3.400E-06*units.m)).to(units.Hz).value

In [ ]:
(constants.c/3.400E-06/units.m).to(units.Hz).value

In [ ]:
(constants.c/5.468E-07/units.m).to(units.Hz).value

## plot vLv-v

In [ ]:
#mjd_list=[54467,55370,58330,58350,58380,58483,58695,58829]
mjd_list=[54467.73,55381.84,56930.51,56950.76,58323.93,58330.99,58483.19,58681.29,5848358695.23]

markers_pd=['*','.','+','d','h','v','^','x','>','1','2','3','4','8','p','P','<','H','X','D','|','_']
colors_pd=['red','green','blue','yellow','lightblue','cyan','tan','gold','sandybrown','olivedrab','chartreuse','darkgreen','m','skyblue','violet','crimson','purple','magenta','lime','turquoise','lightcyan','teal','indigo']
marker_dict = dict(zip(mjd_list, markers_pd[:len(mjd_list)]))
colors_8 = cm.get_cmap('tab10', 8)
colors_pd_8=[colors_8(0),colors_8(1),colors_8(2),colors_8(3),colors_8(4),colors_8(5),colors_8(6),colors_8(7),]
color_dict=dict(zip(mjd_list, colors_pd[:len(mjd_list)]))

def plot_xrt_sed_time(mjd,data_xrt_goodfit_NGC1566,marker,color='r'):
    mjd_near_i=np.argmin(abs(data_xrt_goodfit_NGC1566['obs_time']-mjd))
    mjd_near=data_xrt_goodfit_NGC1566['obs_time'][mjd_near_i]
    if abs(mjd_near-mjd)<0.5:
        print('xrt',mjd_near)
        for i in range(5):
            #print(i+1,data_xrt_goodfit_NGC1566.columns[i+10])
            v_i=(i+1)*2*v_1keV
            f_v_i=data_xrt_goodfit_NGC1566[data_xrt_goodfit_NGC1566.columns[i+10]][mjd_near_i]
            #print("%10.3e"% (v_i),"%10.3e"% (f_v_i))
            axs.scatter(v_i,4*np.pi*D_cm**2*v_i*f_v_i,marker=marker,label=mjd,color=color)
            #axs.text(v_i,v_i*f_v_i,str((i+1)*2))
            
            
def plot_uvot_sed_time(mjd,UV_namelist,UV_datalist,UV_lambda_list,marker,color='r'):
    for i,j,l in zip(UV_namelist,UV_datalist,UV_lambda_list):
        mjd_near_i=np.argmin(abs(j[j.columns[3]]-mjd))
        mjd_near=j[j.columns[3]][mjd_near_i]           
        if abs(mjd_near-mjd)<1.0:
            v_i=(constants.c/l/units.m).to(units.Hz).value
            f_v_i=j[j.columns[-1]][mjd_near_i]  
            print(i,mjd_near)
            #print("%10.3e"% (v_i),"%10.3e"% (f_v_i))
            axs.scatter(v_i,4*np.pi*D_cm**2*v_i*f_v_i,marker=marker,label=mjd,color=color) 
            #axs.text(v_i,v_i*f_v_i,i)

def plot_wise_sed_time(mjd,wise_namelist,wise_datalist,wise_lambdalist,marker,color='r'):
    for i,j,l in zip(wise_namelist,wise_datalist,wise_lambdalist):
        mjd_near_i=np.argmin(abs(j[j.columns[0]]-mjd))
        mjd_near=j[j.columns[0]][mjd_near_i]           
        if abs(mjd_near-mjd)<20:
            print(i,mjd_near)
            v_i=(constants.c/l/units.m).to(units.Hz).value
            f_v_i=j[j.columns[3]][mjd_near_i]
            print(i,"%10.3e"% (v_i),"%10.3e"% (f_v_i))
            axs.scatter(v_i,4*np.pi*D_cm**2*v_i*f_v_i,marker=marker,label=mjd,color=color) 
            #axs.text(v_i,v_i*f_v_i,i[:2])
            
def plot_mjd_sed(mjd,marker,color):
    plot_uvot_sed_time(mjd,UV_namelist,UV_datalist,UV_lambda_list,marker,color)
    plot_xrt_sed_time(mjd,data_xrt_goodfit_NGC1566,marker,color)
    if mjd < 56000:
        plot_wise_sed_time(mjd,multiwise_namelist,multiwise_datalist,wise_lambdalist,marker,color)                
    else:
        plot_wise_sed_time(mjd,neowise_namelist,neowise_datalist,wise_lambdalist,marker,color)
          

fig, axs = plt.subplots(1, 1,figsize=(8,8))
axs.set_title('NGC1566_SED')
axs.semilogy()
axs.semilogx()
axs.set_ylabel(r'$\nu L_{\nu}$ ($erg s^{-1} cm^{-2}$)')
axs.set_xlabel('Hz')

for i in mjd_list:
    marker=marker_dict[i]
    color=color_dict[i]
    plot_mjd_sed(i,marker,color)

    
axs.xaxis.set_tick_params(which='major', size=10, width=2, direction='in', top='on')
axs.xaxis.set_tick_params(which='minor', size=7, width=2, direction='in', top='on')
axs.yaxis.set_tick_params(which='major', size=10, width=2, direction='in', right='on')
axs.yaxis.set_tick_params(which='minor', size=7, width=2, direction='in', right='on')
#axs.xaxis.set_major_locator(mpl.ticker.MultipleLocator(100))
#axs.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(10))
    
#axs.set_xlim(6.0e13,1.0e19)    
    
handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))    
axs.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(1.2, 0.7),ncol=1,loc='right')    
axs.grid(alpha=0.1,which='major', linestyle='--', linewidth=1)
plt.tight_layout()
plt.savefig('/Users/brettlv/Downloads/NGC1566/NGC1566_sed_vLv-v-based_on_swift.png',dpi=400)

In [ ]:
D_cm

(D ≈ 7.2 Mpc, see comments in Paper I) NGC 1566
We estimate Eddington ratios for the minimum (2014) and maximum (2018) using the 2–10 keV fluxes and a mass of ∼ 10^6.9 M⊙ (Woo & Urry 2002), a distance of ∼7.2 Mpc 
and taking a bolometric correction factor of 20